# Introduction


*   While analyzing the predictions made by the initial BERT model, we noticed that **some 'Neutral' comments were actually not 'Neutral'**
*   It appears that this class has been used like a **"trash" class** when manual labeling was difficult
*   Some samples were labeled with multiple labels, including the 'Neutral' emotion. However, **a sample cannot be 'Neutral' and also include other emotions**
*   Training a model on these **noisy data** could lead to increase the confusion between some emotions
* Let's try and train a new model by **filtering out the 'Neutral'** samples



# 1 - Importing libraries and loading data

## 1.1 - Installing and importing libraries

First, let's install the `transformers` library which contains thousands of pre-trained models, including BERT.

In [ ]:
!pip install transformers
!pip install emoji
!pip install contractions

In [ ]:
# Data manipulation libraries
import sys, os
import pandas as pd
import numpy as np
import json

import emoji
import contractions
import re

# Scikit-learn packages
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight

# Packages to define a BERT model
from transformers import TFBertModel, BertTokenizerFast, BertConfig

# Keras and TensorFlow packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.2 - Loading datasets and lists of emotions

First, let's load our clean data.

In [ ]:
# Importing train, validation and test datasets with preprocessed texts and labels
train_GE = pd.read_csv("/content/drive/MyDrive/GoEmotions_Git/data/train_clean.csv")
val_GE = pd.read_csv("/content/drive/MyDrive/GoEmotions_Git/data/val_clean.csv")
test_GE = pd.read_csv("/content/drive/MyDrive/GoEmotions_Git/data/test_clean.csv")

# Shape validation
print(train_GE.shape)
print(val_GE.shape)
print(test_GE.shape)

(43410, 29)
(5426, 29)
(5427, 29)


Let's also load the lists of emotions from GoEmotions and Ekman taxonomies **excluding the the 'Neutral' emotion** this time.

In [ ]:
# Loading emotion labels for GoEmotions taxonomy
with open("/content/drive/MyDrive/GoEmotions_Git/data/emotions.txt", "r") as file:
    GE_taxonomy = file.read().split("\n")
GE_taxonomy.remove('neutral')
print("Emotions on GoEmotions taxonomy are : \n{}".format(GE_taxonomy))

print()

# Loading emotion labels for Ekman taxonomy
with open("/content/drive/MyDrive/GoEmotions_Git/data/ekman_labels.txt", "r") as file:
    Ekman_taxonomy = file.read().split("\n")
Ekman_taxonomy.remove('neutral')
print("Emotions on Ekman taxonomy are : \n{}".format(Ekman_taxonomy))

Emotions on GoEmotions taxonomy are : 
['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']

Emotions on Ekman taxonomy are : 
['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']


## 1.3 - Filtering out the 'Neutral' only samples

First, we need to drop the 'Neutral' emotion from all datasets.

In [ ]:
train_GE = train_GE.drop(columns=['neutral'])
val_GE = val_GE.drop(columns=['neutral'])
test_GE = test_GE.drop(columns=['neutral'])

Then, we need remove all the samples that have been left without a label.

In [ ]:
# Removing samples with only 0 in their labels
train_GE = train_GE.loc[ train_GE.apply(lambda x: sum(x[1:]), axis=1)>0 ]
val_GE = val_GE.loc[ val_GE.apply(lambda x: sum(x[1:]), axis=1)>0 ]
test_GE = test_GE.loc[ test_GE.apply(lambda x: sum(x[1:]), axis=1)>0 ]

# Shape validation
print(train_GE.shape)
print(val_GE.shape)
print(test_GE.shape)

(30587, 28)
(3834, 28)
(3821, 28)


In [ ]:
# Preview of data
display(train_GE.head(3))

,Clean_text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise
2,why the fuck is bayless isoing,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,to make her feel threatened,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,dirty southern wankers,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Doing so, we have **decreased the number of samples by nearly 30%** of the original data.

#2 - Modeling : BERT (Bidirectional Encoder Representations from Transformers)

Now we can go ahead and start defining our BERT-based model.

##2.1 - Configuration of the base model

First of all, let's define a `max_length` variable. This variable sets a fixed length of sequences to be fed to our model. Therefore, sequences will be either truncated if larger than this value, or completed using padding if smaller. To avoid truncating, we fix this value according to the largest sample of our data.

In [ ]:
# Computing max length of samples
full_text = pd.concat([train_GE['Clean_text'], val_GE['Clean_text'], test_GE['Clean_text']])
max_length = full_text.apply(lambda x: len(x.split())).max()
max_length

48

We are going to use BERT's base model which contains almost 110 M trainable parameters. 

Also, in order to match the tokenization and vocabulary used during the training, we are going to use a BERT tokenizer.

In [ ]:
# Importing BERT pre-trained model and tokenizer
model_name = 'bert-base-uncased'
config = BertConfig.from_pretrained(model_name, output_hidden_states=False)
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


## 2.2 - Definition of the model architecture

Now that everything is in place, we can create a model based on BERT's main layer, and replace the top layers to reach our main objective (multi-label classification accross **27 possible emotions**).

Our model takes three inputs that result from tokenization:

*   `input_ids`: indices of input sequence tokens in the vocabulary
*   `token_ids`: Segment token indices to indicate first and second portions of the inputs.   0 for sentence A and 1 for sentence B
*   `attention mask`: Mask to avoid performing attention on padding token indices.  0 for masked and 1 for not masked



In [ ]:
# function for creating BERT based model
def create_model(nb_labels):

  # Load the MainLayer
  bert = transformer_model.layers[0]

  # Build the model inputs
  input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
  attention_mask = Input(shape=(max_length,), name='attention_mask', dtype='int32')
  token_ids = Input(shape=(max_length,), name='token_ids', dtype='int32')
  inputs = {'input_ids': input_ids, 'attention_mask': attention_mask, 'token_ids': token_ids}

  # Load the Transformers BERT model as a layer in a Keras model
  bert_model = bert(inputs)[1]
  dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
  pooled_output = dropout(bert_model, training=False)

  # Then build the model output
  emotion = Dense(units=nb_labels, activation="sigmoid", kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='emotion')(pooled_output)
  outputs = emotion

  # And combine it all in a model object
  model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel')

  return model

We use here a `sigmoid` activation function in the last dense layer that is better suited than a `softmax` activation function. In fact, `softmax` shrinks output probabilities for each label so that the sum of probabilities is 1. In our case, each label (emotion) can independently have a probability between 0 and 1, and `sigmoid` allows that.

We can now create our model using 27 labels and visualize a summary.

In [ ]:
# Creating a model instance
model = create_model(27)

# Take a look at the model
model.summary()

Model: "BERT_MultiLabel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
attention_mask (InputLayer)     [(None, 48)]         0                                            
__________________________________________________________________________________________________
input_ids (InputLayer)          [(None, 48)]         0                                            
__________________________________________________________________________________________________
token_ids (InputLayer)          [(None, 48)]         0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 109482240   attention_mask[0][0]             
                                                                 input_ids[0][0]    

##2.3 - Data preprocessing and model training

###2.3.1 - Tokenizing data

Let's go ahead and process our data. We will first separate texts from labels in the train, validation and test datasets, and then tokenize the texts using the BERT tokenizer.

In [ ]:
# Creating train, validation and test variables
X_train = train_GE['Clean_text']
y_train = train_GE.loc[:, GE_taxonomy].values.astype(float)

X_val = val_GE['Clean_text']
y_val = val_GE.loc[:, GE_taxonomy].values.astype(float)

X_test = test_GE['Clean_text']
y_test = test_GE.loc[:, GE_taxonomy].values.astype(float)

# Tokenizing train data
train_token = tokenizer(
    text = X_train.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length', 
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

# Tokenizing valisation data
val_token = tokenizer(
    text = X_val.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length', 
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

# Tokenizing test data
test_token = tokenizer(
    text = X_test.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length', 
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

In [ ]:
# Creating BERT compatible inputs with Input Ids, attention masks and token Ids 
train = {'input_ids': train_token['input_ids'], 'attention_mask': train_token['attention_mask'],'token_ids': train_token['token_type_ids']}
val = {'input_ids': val_token['input_ids'], 'attention_mask': val_token['attention_mask'],'token_ids': val_token['token_type_ids']}
test = {'input_ids': test_token['input_ids'], 'attention_mask': test_token['attention_mask'],'token_ids': test_token['token_type_ids']}

During the training phase, we our going to use batches of 16 samples. After each epoch, data will be shuffled. Let's create TensorFlow tensors accordingly.

In [ ]:
# Creating TF tensors
train_tensor = tf.data.Dataset.from_tensor_slices((train, y_train)).shuffle(len(train)).batch(16)
val_tensor = tf.data.Dataset.from_tensor_slices((val, y_val)).shuffle(len(val)).batch(16)
test_tensor = tf.data.Dataset.from_tensor_slices((test, y_test)).shuffle(len(test)).batch(16)

### 2.3.2 - Class weights for multi-label and custom loss function

Training requires to monitor the loss function and eventually some other metrics to see how the model behaves throughout the epochs.

Therefore, we need to define a weighted loss function that takes into account  class weights in our multi-label case.

First, we need to compute class weights.

In [ ]:
# Function for calculating multilabel class weights
def calculating_class_weights(y_true):
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight('balanced', [0.,1.], y_true[:, i])
    return weights

class_weights = calculating_class_weights(y_train)

Then, we can define a custom crossentropy function in which we multiply the weights.

In [ ]:
# Custom loss function for multilabel
def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss

###2.3.3 - Model training

Everything is ready, we can now start training our model.

We chose not to exceed 4 epochs to train our model as it will most likely start to overfit our data.

In [ ]:
# Set an optimizer
optimizer = Adam(
    learning_rate=3.e-05,
    )

# Set loss
loss = get_weighted_loss(class_weights)

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss)

# train the model
history = model.fit(train_tensor, 
                    epochs=4, 
                    validation_data=val_tensor,
                    )

Epoch 1/4
1912/1912 [==============================] - 359s 180ms/step - loss: 0.5433 - val_loss: 0.3391
Epoch 2/4
1912/1912 [==============================] - 342s 179ms/step - loss: 0.3039 - val_loss: 0.2979
Epoch 3/4
1912/1912 [==============================] - 342s 179ms/step - loss: 0.2258 - val_loss: 0.3288
Epoch 4/4
1912/1912 [==============================] - 341s 178ms/step - loss: 0.1726 - val_loss: 0.3570


## 2.4 - Model evaluation

### 2.4.1 - Evaluation on GoEmotions taxonomy

In [ ]:
# Save model weights
model.save_weights('/content/drive/MyDrive/GoEmotions_Git/bert-weights.hdf5')

Let's generate predictions on test data.

In [ ]:
# Making probability predictions on test data
y_pred_proba = model.predict(test)

When making predictions, we only generate probabilities associated with each label. To predict actual labels, we need to add an additional step that transforms these probabilities into labels given a certain threshold.

We define a function to do so with a default threshold set to 0.8.

In [ ]:
# from probabilities to labels using a given threshold
def proba_to_labels(y_pred_proba, threshold=0.8):
    
    y_pred_labels = np.zeros_like(y_pred_proba)
    
    for i in range(y_pred_proba.shape[0]):
        for j in range(y_pred_proba.shape[1]):
            if y_pred_proba[i][j] > threshold:
                y_pred_labels[i][j] = 1
            else:
                y_pred_labels[i][j] = 0
                
    return y_pred_labels

In [ ]:
# Generate labels
y_pred_labels = proba_to_labels(y_pred_proba)

Let's evaluate these predictions using the evaluation function we defined in the previous notebooks.

In [ ]:
# Model evaluation function 
def model_eval(y_true, y_pred_labels, emotions):
    
    # Defining variables
    precision = []
    recall = []
    f1 = []
    
    # Per emotion evaluation      
    idx2emotion = {i: e for i, e in enumerate(emotions)}
    
    for i in range(len(emotions)):
   
        # Computing precision, recall and f1-score
        p, r, f1_score, _ = precision_recall_fscore_support(y_true[:, i], y_pred_labels[:, i], average="binary")
        
        # Append results in lists
        precision.append(round(p, 2))
        recall.append(round(r, 2))
        f1.append(round(f1_score, 2))
    
    # Macro evaluation
    macro_p, macro_r, macro_f1_score, _ = precision_recall_fscore_support(y_true, y_pred_labels, average="macro")
    
    # Append results in lists
    precision.append(round(macro_p, 2))
    recall.append(round(macro_r, 2))
    f1.append(round(macro_f1_score, 2))
    
    # Converting results to a dataframe
    df_results = pd.DataFrame({"Precision":precision, "Recall":recall, 'F1':f1})
    df_results.index = emotions+['MACRO-AVERAGE']
    
    return df_results

In [ ]:
# Model evaluation
model_eval(y_test, y_pred_labels, GE_taxonomy)

,Precision,Recall,F1
admiration,0.74,0.68,0.71
amusement,0.77,0.92,0.84
anger,0.43,0.69,0.53
annoyance,0.37,0.52,0.43
approval,0.46,0.51,0.48
caring,0.38,0.64,0.47
confusion,0.32,0.82,0.46
curiosity,0.54,0.86,0.67
desire,0.45,0.63,0.53
disappointment,0.35,0.36,0.36


Looking at the results, we see that this model performs better than the previous one. It looks like **removing the noise brought by the 'Neutral' emotion helped to better distinguish the other emotions.**

### 2.4.2 - Threshold optimization

In the initial evaluation, we set an aribitrary threshold. However, we can also choose a threshold that maximizes a certain metric. 

We define a function that tests a certain number of possible thresholds, and returns the best threshold together with the best predicted labels and best macro f1-score.

In [ ]:
# Function that computes labels from probabilities and optimizes the threshold that maximizes f1-score
def proba_to_labels_opt(y_true, y_pred_proba):
    
    '''
    Inputs:
        y_true: Ground truth labels
        y_pred_proba: predicted probabilities
        
    Outputs :
        best_y_pred_labels: preticted labels associated with best threshold
        best_t: best threshold
        best_macro_f1: macro f1-score associated with predicted labels
    '''
    
    # range of possible thresholds
    thresholds = np.arange(0.7, 0.99, 0.01)
    
    # Computing threshold that maximizes macro f1-score 
    best_y_pred_labels = np.zeros_like(y_pred_proba)
    best_t = 0
    best_macro_f1 = 0
    
    # Iterating through possible thresholds
    for t in thresholds:
        
        y_pred_labels = proba_to_labels(y_pred_proba, t)
                             
        _, _, macro_f1, _ = precision_recall_fscore_support(y_true, y_pred_labels, average="macro")
        
        if macro_f1 > best_macro_f1:
            best_macro_f1 = macro_f1
            best_t = t
            best_y_pred_labels = y_pred_labels
            
    return best_y_pred_labels, best_t, best_macro_f1

We can now apply this function to our predicted probabilities and compute optimized label predictions.

In [ ]:
# Compute label predictions and corresponding optimal thresholds 
y_pred_labels_opt, threshold_opt, macro_f1_opt = proba_to_labels_opt(y_test, y_pred_proba)
print("The model's threshold is {}".format(threshold_opt))
print("The model's best macro-f1 is {}".format(macro_f1_opt))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The model's threshold is 0.8700000000000001
The model's best macro-f1 is 0.5292056523991163


In [ ]:
# Model evaluation : Precision, Recall, F-score
model_eval(y_test, y_pred_labels_opt, GE_taxonomy)

,Precision,Recall,F1
admiration,0.78,0.62,0.69
amusement,0.80,0.90,0.84
anger,0.49,0.63,0.55
annoyance,0.47,0.34,0.40
approval,0.51,0.43,0.46
caring,0.43,0.59,0.50
confusion,0.38,0.76,0.51
curiosity,0.59,0.81,0.68
desire,0.57,0.61,0.59
disappointment,0.44,0.29,0.35


**Optimizing the threshold** helped us to **slightly improve** the model predictions.

### 2.4.4 - Indirect evaluation on Ekman taxonomy by mapping predictions

Until now, we have only evaluated our model on the GoEmotions taxonomy.

As a reference, we can try to map the true and predicted emotions to the Ekman taxonomy and see how our model performs.

We have already defined the Ekman taxonomy earlier.

Let's define a function that transforms labels from GoEmotions to Ekman taxonomy.

In [ ]:
# Function thats maps predictions on GoEmotions taxonomy to Ekman taxonomy
def GE_to_Ekman(GE_labels):
    
    # Create a dataframe of GoEmotions labels
    df_GE = pd.DataFrame(GE_labels, columns=GE_taxonomy)

    # Create an empty dataframe of Ekman labels
    df_Ekman  = pd.DataFrame(np.zeros((len(GE_labels), len(Ekman_taxonomy))), columns=Ekman_taxonomy)

    for i in range(len(df_GE)):

        if df_GE.loc[i,['anger', 'annoyance', 'disapproval']].sum() >= 1:
            df_Ekman.loc[i,'anger'] = 1

        if df_GE.loc[i,'disgust'].sum() >= 1:
            df_Ekman.loc[i,'disgust'] = 1

        if df_GE.loc[i,['fear', 'nervousness']].sum() >= 1:
            df_Ekman.loc[i,'fear'] = 1

        if df_GE.loc[i,['joy', 'amusement', 'approval', 'excitement', 'gratitude',
                        'love', 'optimism', 'relief', 'pride', 'admiration', 'desire','caring']].sum() >= 1:
            df_Ekman.loc[i,'joy'] = 1 

        if df_GE.loc[i,['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse']].sum() >= 1:
            df_Ekman.loc[i,'sadness'] = 1

        if df_GE.loc[i,['surprise', 'realization', 'confusion', 'curiosity']].sum() >= 1:
            df_Ekman.loc[i,'surprise'] = 1

    return df_Ekman.values

We can now apply our function and evaluate the predictions

In [ ]:
# Mapping GoEmotion labels to Ekman labels (true and predictions)
y_test_Ekman = GE_to_Ekman(y_test)
y_pred_labels_Ekman = GE_to_Ekman(y_pred_labels_opt)

# Evaluation
model_eval(y_test_Ekman, y_pred_labels_Ekman, Ekman_taxonomy)

,Precision,Recall,F1
anger,0.71,0.63,0.67
disgust,0.37,0.71,0.49
fear,0.60,0.85,0.70
joy,0.90,0.88,0.89
sadness,0.66,0.63,0.64
surprise,0.65,0.75,0.70
MACRO-AVERAGE,0.65,0.74,0.68


## 2.5 - Make predictions

To make predictions on a new sample, it needs to be processed using all the different precessing steps we used.

In [ ]:
# Retrieving initial preprocessings
def preprocess_corpus(x):
    
    # Adding a space between words and punctation
    x = re.sub( r'([a-zA-Z\[\]])([,;.!?])', r'\1 \2', x)
    x = re.sub( r'([,;.!?])([a-zA-Z\[\]])', r'\1 \2', x)

    # Demojize
    x = emoji.demojize(x)

    # Expand contraction
    x = contractions.fix(x)

    # Lower
    x = x.lower()

    #correct some acronyms/typos/abbreviations  
    x = re.sub(r"lmao", "laughing my ass off", x)  
    x = re.sub(r"amirite", "am i right", x)
    x = re.sub(r"\b(tho)\b", "though", x)
    x = re.sub(r"\b(ikr)\b", "i know right", x)
    x = re.sub(r"\b(ya|u)\b", "you", x)
    x = re.sub(r"\b(eu)\b", "europe", x)
    x = re.sub(r"\b(da)\b", "the", x)
    x = re.sub(r"\b(dat)\b", "that", x)
    x = re.sub(r"\b(dats)\b", "that is", x)
    x = re.sub(r"\b(cuz)\b", "because", x)
    x = re.sub(r"\b(fkn)\b", "fucking", x)
    x = re.sub(r"\b(tbh)\b", "to be honest", x)
    x = re.sub(r"\b(tbf)\b", "to be fair", x)
    x = re.sub(r"faux pas", "mistake", x)
    x = re.sub(r"\b(btw)\b", "by the way", x)
    x = re.sub(r"\b(bs)\b", "bullshit", x)
    x = re.sub(r"\b(kinda)\b", "kind of", x)
    x = re.sub(r"\b(bruh)\b", "bro", x)
    x = re.sub(r"\b(w/e)\b", "whatever", x)
    x = re.sub(r"\b(w/)\b", "with", x)
    x = re.sub(r"\b(w/o)\b", "without", x)
    x = re.sub(r"\b(doj)\b", "department of justice", x)

    # replace some words with multiple occurences of a letter, example "coooool" turns into --> cool
    x = re.sub(r"\b(j+e{2,}z+e*)\b", "jeez", x)
    x = re.sub(r"\b(co+l+)\b", "cool", x)
    x = re.sub(r"\b(g+o+a+l+)\b", "goal", x)
    x = re.sub(r"\b(s+h+i+t+)\b", "shit", x)
    x = re.sub(r"\b(o+m+g+)\b", "omg", x)
    x = re.sub(r"\b(w+t+f+)\b", "wtf", x)
    x = re.sub(r"\b(w+h+a+t+)\b", "what", x)
    x = re.sub(r"\b(y+e+y+|y+a+y+|y+e+a+h+)\b", "yeah", x)
    x = re.sub(r"\b(w+o+w+)\b", "wow", x)
    x = re.sub(r"\b(w+h+y+)\b", "why", x)
    x = re.sub(r"\b(s+o+)\b", "so", x)
    x = re.sub(r"\b(f)\b", "fuck", x)
    x = re.sub(r"\b(w+h+o+p+s+)\b", "whoops", x)
    x = re.sub(r"\b(ofc)\b", "of course", x)
    x = re.sub(r"\b(the us)\b", "usa", x)
    x = re.sub(r"\b(gf)\b", "girlfriend", x)
    x = re.sub(r"\b(hr)\b", "human ressources", x)
    x = re.sub(r"\b(mh)\b", "mental health", x)
    x = re.sub(r"\b(idk)\b", "i do not know", x)
    x = re.sub(r"\b(gotcha)\b", "i got you", x)
    x = re.sub(r"\b(y+e+p+)\b", "yes", x)
    x = re.sub(r"\b(a*ha+h[ha]*|a*ha +h[ha]*)\b", "haha", x)
    x = re.sub(r"\b(o?l+o+l+[ol]*)\b", "lol", x)
    x = re.sub(r"\b(o*ho+h[ho]*|o*ho +h[ho]*)\b", "ohoh", x)
    x = re.sub(r"\b(o+h+)\b", "oh", x)
    x = re.sub(r"\b(a+h+)\b", "ah", x)
    x = re.sub(r"\b(u+h+)\b", "uh", x)

    # Handling emojis
    x = re.sub(r"<3", " love ", x)
    x = re.sub(r"xd", " smiling_face_with_open_mouth_and_tightly_closed_eyes ", x)
    x = re.sub(r":\)", " smiling_face ", x)
    x = re.sub(r"^_^", " smiling_face ", x)
    x = re.sub(r"\*_\*", " star_struck ", x)
    x = re.sub(r":\(", " frowning_face ", x)
    x = re.sub(r":\^\(", " frowning_face ", x)
    x = re.sub(r";\(", " frowning_face ", x)
    x = re.sub(r":\/",  " confused_face", x)
    x = re.sub(r";\)",  " wink", x)
    x = re.sub(r">__<",  " unamused ", x)
    x = re.sub(r"\b([xo]+x*)\b", " xoxo ", x)
    x = re.sub(r"\b(n+a+h+)\b", "no", x)
    
    # Handling special cases of text
    x = re.sub(r"h a m b e r d e r s", "hamberders", x)
    x = re.sub(r"b e n", "ben", x)
    x = re.sub(r"s a t i r e", "satire", x)
    x = re.sub(r"y i k e s", "yikes", x)
    x = re.sub(r"s p o i l e r", "spoiler", x)
    x = re.sub(r"thankyou", "thank you", x)
    x = re.sub(r"a^r^o^o^o^o^o^o^o^n^d", "around", x)

    # Remove special characters and numbers replace by space + remove double space
    x = re.sub(r"\b([.]{3,})"," dots ", x)
    x = re.sub(r"[^A-Za-z!?_]+"," ", x)
    x = re.sub(r"\b([s])\b *","", x)
    x = re.sub(r" +"," ", x)
    x = x.strip()

    return x     

Now we can define a prediction function that takes one or more samples, and outputs the detected emotions from the model.

In [ ]:
def predict_samples(text_samples, model, threshold):
    
    # Text preprocessing and cleaning
    text_samples_clean = [preprocess_corpus(text) for text in text_samples]
    
    # Tokenizing train data
    samples_token = tokenizer(
        text = text_samples_clean,
        add_special_tokens = True,
        max_length = max_length,
        truncation = True,
        padding = 'max_length', 
        return_tensors = 'tf',
        return_token_type_ids = True,
        return_attention_mask = True,
        verbose = True,
    )
    
    # Preparing to feed the model
    samples = {'input_ids': samples_token['input_ids'],
               'attention_mask': samples_token['attention_mask'],
               'token_ids': samples_token['token_type_ids']
              }
    
    # Probability predictions
    samples_pred_proba = model.predict(samples)
    
    # Label prediction using threshold
    samples_pred_labels = proba_to_labels(samples_pred_proba)
            
    samples_pred_labels_df = pd.DataFrame(samples_pred_labels)
    samples_pred_labels_df = samples_pred_labels_df.apply(lambda x: [GE_taxonomy[i] for i in range(len(x)) if x[i]==1], axis=1)
    
    #return list(samples_pred_labels_df)
    return pd.DataFrame({"Text":text_samples, "Emotions":list(samples_pred_labels_df)})

Let's try on few examples.

In [ ]:
# Predict samples
predict_samples(["My favourite food is anything I didn't have to cook myself", "are you kiddin me ??!!", "red"], model, threshold_opt)

,Text,Emotions
0,My favourite food is anything I didn't have to...,[joy]
1,are you kiddin me ??!!,"[confusion, curiosity]"
2,red,[]


Although the score is not very high, we see that the model **detects emotions that coherent**. Also, **when entering a neutral text such as "red", the model does not detect any emotion**. 

# Conclusion

*   In this notebook, we verified our assumption : **filtering out the 'Neutral' samples from the data allows to improve the model**.

*   Not only **we better distinguish actual emotions**, but **we can also detect 'Neutral' comments**, **without teaching our model what is a 'Neutral' comment**.